In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
!pip install osmnx
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, GeometryCollection

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 1.9 MB/s eta 0:00:00


In [4]:
if os.path.exists('/content/drive/My Drive/Capstone/Data'):
    base_dir = '/content/drive/My Drive/Capstone/Data'
else:
    base_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'Data'))

In [5]:
stops = gpd.read_file(os.path.join(base_dir, "yandex_scraped/full_stops.geojson"))

unique_stops = stops.drop_duplicates(subset=['geometry'])
unique_stops = unique_stops.reset_index(drop=True)

In [6]:
ox.settings.log_console = True
ox.settings.use_cache = True

place_name = "Yerevan, Armenia"
G_walk = ox.graph_from_place(place_name, network_type='walk')

In [7]:
def generate_isochrone(G_walk, stops_gdf, time_minutes):
    """
    Generates isochrone polygons for each public transport stop within a specified walking time.

    Args:
        G_walk (networkx.MultiDiGraph): A walkable street network graph (typically from OSMnx).
        stops_gdf (GeoDataFrame): GeoDataFrame of stops with geometry in latitude/longitude.
        time_minutes (int or float): Time in minutes to define the isochrone (e.g., 5, 10, 15).

    Returns:
        GeoDataFrame: A GeoDataFrame where each row is a stop with its associated isochrone polygon,
                      distance threshold (in meters), and time value.
    """
    walking_speed = 1.4
    distance_m = walking_speed * time_minutes * 60

    G_proj = ox.project_graph(G_walk)
    nodes_proj = ox.graph_to_gdfs(G_proj, nodes=True, edges=False)
    stops_proj = stops_gdf.to_crs(nodes_proj.crs)

    records = []

    for idx, stop in stops_proj.iterrows():
        stop_point = stop.geometry
        stop_id = idx
        nearest_node = ox.distance.nearest_nodes(G_proj, X=stop_point.x, Y=stop_point.y)

        subgraph_lengths = nx.single_source_dijkstra_path_length(G_proj, nearest_node, cutoff=distance_m, weight='length')
        reachable_nodes = [node for node, dist in subgraph_lengths.items() if dist <= distance_m]

        if reachable_nodes:
            reachable_points = nodes_proj.loc[reachable_nodes].geometry
            isochrone_polygon = reachable_points.unary_union.convex_hull

            records.append({
                "stop_id": stop_id,
                "time_min": time_minutes,
                "distance_m": distance_m,
                "geometry": isochrone_polygon
            })

    isochrones_gdf = gpd.GeoDataFrame(records, crs=stops_proj.crs)
    return isochrones_gdf

In [8]:
iso = generate_isochrone(G_walk, unique_stops, 5)
iso_4326 = iso.to_crs(epsg=4326)

<ipython-input-7-31b53c118153>:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  isochrone_polygon = reachable_points.unary_union.convex_hull
<ipython-input-7-31b53c118153>:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  isochrone_polygon = reachable_points.unary_union.convex_hull
<ipython-input-7-31b53c118153>:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  isochrone_polygon = reachable_points.unary_union.convex_hull
<ipython-input-7-31b53c118153>:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  isochrone_polygon = reachable_points.unary_union.convex_hull
<ipython-input-7-31b53c118153>:33: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  isochrone_polygon = reachable_points.unary_union.convex_hull
<ipyt

KeyboardInterrupt: 

In [ ]:
def clean_and_union_isochrones(iso_gdf):
    """
    Cleans a GeoDataFrame of isochrones by removing empty geometries and unions all valid polygons
    into a single (Multi)Polygon per time category.

    Args:
        iso_gdf (GeoDataFrame): GeoDataFrame containing individual isochrone geometries.

    Returns:
        GeoDataFrame: A single-row GeoDataFrame with a unified isochrone polygon and the associated time.
    """
    iso_clean = iso_gdf[~iso_gdf['geometry'].is_empty].copy()
    unioned_geometry = iso_clean.unary_union

    if isinstance(unioned_geometry, GeometryCollection):
        polygons = [
            geom for geom in unioned_geometry.geoms
            if geom.geom_type in ['Polygon', 'MultiPolygon']
        ]
        cleaned_geometry = MultiPolygon(polygons)
    else:
        cleaned_geometry = unioned_geometry

    return gpd.GeoDataFrame(
        [{'geometry': cleaned_geometry, 'time_min': iso_clean['time_min'].iloc[0]}],
        crs=iso_clean.crs
    )

In [ ]:
iso_union = clean_and_union_isochrones(iso_4326)
iso_4326.to_file(os.path.join(base_dir, "isochrones/isochrones_5min.geojson"), driver='GeoJSON')
iso_union.to_file(os.path.join(base_dir, "isochrones/isochrones_5min_union.geojson"), driver='GeoJSON')